<a href="https://colab.research.google.com/github/Hidetsune/SemEval2024_Task4/blob/main/SemEval_2024_Task_4_Bulgarian_Hidetsune_Takahashi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import and Dataset Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json

In [ ]:
import re

In [ ]:
training_validation_data = pd.read_csv("/content/drive/MyDrive/SemEval Task4/task4_data_concat_extended5.csv") # task5_data_concat_extended4.csv   (CHANGE HERE)
training_validation_data

,text,propagandistic/non_propagandistic
0,we're here to help! send us a dm with your em...,non_propagandistic
1,then now raise your hand if jill should res...,propagandistic
2,"hello, this does not sound good. can you dm ...",non_propagandistic
3,"sorry for the delay, katie! we'd like to inve...",non_propagandistic
4,you should have to pass a drug test to receive...,propagandistic
...,...,...
20769,writing for facerook tio apoloigai td mlrldott...,propagandistic
20770,"hey matthew, flight 783 is showing a late arr...",non_propagandistic
20771,"if the permissions cannot be disabled, they m...",non_propagandistic
20772,isn't prime two day shipping supposed to be g...,non_propagandistic


In [ ]:
!pip install "spacy >= 3.0.6"
!pip install "comet_ml>=3.31.19"
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.4/599.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.7/514.7 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.7.0
    Uninstalling websocket-client-1.7.0:
      Successfully uninstalled websocket-client-1.7.0
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.1 MB/s 

In [ ]:
from datasets import Dataset, Value, ClassLabel, Features  #added

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
training_data, validation_data = train_test_split(training_validation_data, test_size=0.05, random_state=10)

In [ ]:
dataset_train = Dataset.from_pandas(training_data)
dataset_val = Dataset.from_pandas(validation_data)

In [ ]:
categories = ['propagandistic', 'non_propagandistic']

In [ ]:
new_features = dataset_train.features.copy()

In [ ]:
new_features['text'] = Value('large_string')
new_features['propagandistic/non_propagandistic'] = ClassLabel(names=['propagandistic', 'non_propagandistic'])

In [ ]:
dataset_train = dataset_train.cast(new_features)
dataset_val = dataset_val.cast(new_features)

Casting the dataset:   0%|          | 0/19735 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1039 [00:00<?, ? examples/s]

In [ ]:
dataset_train.features

{'text': Value(dtype='large_string', id=None),
 'propagandistic/non_propagandistic': ClassLabel(names=['propagandistic', 'non_propagandistic'], id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
dataset_val.features

{'text': Value(dtype='large_string', id=None),
 'propagandistic/non_propagandistic': ClassLabel(names=['propagandistic', 'non_propagandistic'], id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

#Training

In [ ]:
from typing import Set, List, Tuple
from spacy.tokens import DocBin
import spacy
from tqdm.auto import tqdm

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

In [ ]:
def convert_spacy_dataset(dataset, target_file: str, labels):
  nlp = spacy.blank("en")
  db = DocBin()

  for item in tqdm(dataset):
    try:
      doc = nlp.make_doc(item["text"])
      doc.cats = {label: 0 for label in labels}
      doc.cats[labels[item["propagandistic/non_propagandistic"]]] = 1

      db.add(doc)
    except:
      continue

  db.to_disk(target_file)
  return db

In [ ]:
dataset_val

Dataset({
    features: ['text', 'propagandistic/non_propagandistic', '__index_level_0__'],
    num_rows: 1039
})

In [ ]:
convert_spacy_dataset(dataset_train, "train_data.spacy", categories)
convert_spacy_dataset(dataset_val, "validation_data.spacy", categories)

  0%|          | 0/19735 [00:00<?, ?it/s]

  0%|          | 0/1039 [00:00<?, ?it/s]

In [ ]:
!python -m spacy init config --lang en --pipeline textcat_multilabel comet_config.cfg  #training model

2024-01-26 00:36:42.083401: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 00:36:42.083457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 00:36:42.084612: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-26 00:36:43.119708: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- 

In [ ]:
from spacy.cli.train import train as spacy_train

config_path = "comet_config.cfg"
output_model_path = "output_models/"

spacy_train(
    config_path,
    output_path=output_model_path,
    overrides={
        "paths.train": "train_data.spacy",
        "paths.dev": "validation_data.spacy",
    },
    use_gpu=0

)


✔ Created output directory: output_models
ℹ Saving to output directory: output_models
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       63.18    0.63
  0     200          43.20       91.73    0.92
  0     400          30.59       93.52    0.94
  0     600          28.42       95.08    0.95
  0     800          23.64       96.15    0.96
  0    1000          21.22       96.59    0.97
  0    1200          18.56       96.97    0.97
  0    1400          18.38       97.39    0.97
  0    1600          16.18       97.67    0.98
  1    1800          14.27       97.88    0.98
  1    2000          12.48       98.07    0.98
  1    2200          12.23       98.21    0.

#Trial

In [ ]:
trained_nlp = spacy.load("output_models/model-best/")

In [ ]:
!iconv -f UTF-8 -t ASCII dev_unlabeled.json

iconv: cannot open input file `dev_unlabeled.json': No such file or directory


In [ ]:
json_open_trial = open("/content/drive/MyDrive/SemEval Task4/bg_subtask2b_test_unlabeled.json","r", encoding="utf-8")
json_load_trial = json.load(json_open_trial)

In [ ]:
trial_data = json_load_trial
trial_data

[{'id': '1',
  'text': 'Когато се събудиш след като цяла\nнощ си се ваксинирала против\nкоронавирус/I\n',
  'image': 'bg_memes_1.png'},
 {'id': 'bg_mk_memes_984',
  'text': 'Какво има в Костя Копейкин: ACE\n\nКакво има в Кирил Петков:\n\n',
  'image': 'bg_mk_memes_984.png'},
 {'id': '4',
  'text': 'КАК СЕ ЧУВСТВАТЕ?\nИЗВЕДНЪЖ УСЕТИХ\nНЕОБЯСНИМА НУЖДА\nДА СИ РЕГИСТРИРАМ WINDOWS-A\n',
  'image': 'bg_memes_4.png'},
 {'id': '41',
  'text': 'НИКОГА НЕ СЪМ ВЯРВАЛ, ЧЕ ОНОВА\nНАЧАЛО НА ВИЦОВЕТЕ ЩЕ СЕ ОКАЖЕ ИСТИНА:\nБАЙ ГАНЬО БЪЛГАРИНА\nИ БАЙ ГАНЬО АМЕРИКАНЕЦА...\nimgflip.com\n',
  'image': 'bg_memes_41.png'},
 {'id': '49',
  'text': 'i\n2 часа след ваксинацията\nPIK\n',
  'image': 'bg_memes_49.png'},
 {'id': '50',
  'text': 'ПРЕДИ ВАКСИНАТА\nСЛЕДВАКСИНАТА\nimgflip.com\n',
  'image': 'bg_memes_50.png'},
 {'id': '52',
  'text': 'ЕC: Няма повишение в броя на\nбежанците от Афганистан\n07.10.2021 | 00:41 мин. | Сваляне\n@meme_рartiq\nДинко от\nЯмбол:\n',
  'image': 'bg_memes_52.png'},
 {'id': '53',

In [ ]:
!pip install -U deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
from deep_translator import GoogleTranslator

In [ ]:
langs_dict = GoogleTranslator().get_supported_languages(as_dict=True)
langs_dict

{'afrikaans': 'af',
 'albanian': 'sq',
 'amharic': 'am',
 'arabic': 'ar',
 'armenian': 'hy',
 'assamese': 'as',
 'aymara': 'ay',
 'azerbaijani': 'az',
 'bambara': 'bm',
 'basque': 'eu',
 'belarusian': 'be',
 'bengali': 'bn',
 'bhojpuri': 'bho',
 'bosnian': 'bs',
 'bulgarian': 'bg',
 'catalan': 'ca',
 'cebuano': 'ceb',
 'chichewa': 'ny',
 'chinese (simplified)': 'zh-CN',
 'chinese (traditional)': 'zh-TW',
 'corsican': 'co',
 'croatian': 'hr',
 'czech': 'cs',
 'danish': 'da',
 'dhivehi': 'dv',
 'dogri': 'doi',
 'dutch': 'nl',
 'english': 'en',
 'esperanto': 'eo',
 'estonian': 'et',
 'ewe': 'ee',
 'filipino': 'tl',
 'finnish': 'fi',
 'french': 'fr',
 'frisian': 'fy',
 'galician': 'gl',
 'georgian': 'ka',
 'german': 'de',
 'greek': 'el',
 'guarani': 'gn',
 'gujarati': 'gu',
 'haitian creole': 'ht',
 'hausa': 'ha',
 'hawaiian': 'haw',
 'hebrew': 'iw',
 'hindi': 'hi',
 'hmong': 'hmn',
 'hungarian': 'hu',
 'icelandic': 'is',
 'igbo': 'ig',
 'ilocano': 'ilo',
 'indonesian': 'id',
 'irish': 'ga

In [ ]:
for utterance in trial_data:
  utterance["text"] = GoogleTranslator(source='bg', target='en').translate(utterance["text"])

In [ ]:
trial_data

[{'id': '1',
  'text': 'When you wake up after being healed\nnight you got vaccinated against\ncoronavirus/I',
  'image': 'bg_memes_1.png'},
 {'id': 'bg_mk_memes_984',
  'text': "What's in Kostya Kopeikin: ACE\n\nWhat's in Kiril Petkov:",
  'image': 'bg_mk_memes_984.png'},
 {'id': '4',
  'text': 'HOW DO YOU FEEL?\nSUDDENLY I FELT\nAN UNEXPLAINABLE NEED\nTO REGISTER WINDOWS-A',
  'image': 'bg_memes_4.png'},
 {'id': '41',
  'text': 'I NEVER BELIEVED THAT\nTHE BEGINNING OF THE JOKES WILL TURN OUT TO BE TRUE:\nBYE GANYO BULGARINA\nAND BYE GANYO THE AMERICAN...\nimgflip.com',
  'image': 'bg_memes_41.png'},
 {'id': '49',
  'text': 'i\n2 hours after vaccination\nPIK',
  'image': 'bg_memes_49.png'},
 {'id': '50',
  'text': 'BEFORE THE VACCINE\nTHE POST-VACCINE\nimgflip.com',
  'image': 'bg_memes_50.png'},
 {'id': '52',
  'text': 'EC: No increase in the number of\nrefugees from Afghanistan\n07.10.2021 | 00:41 min | Download\n@meme_рartiq\nDinko from\nYambol:',
  'image': 'bg_memes_52.png'},
 {'

In [ ]:
"""
for utterance in trial_data:
  d = trained_nlp(utterance["text"]).cats
  label = max(d, key =d.get)
  utterance["label"] = label
"""

'\nfor utterance in trial_data:\n  d = trained_nlp(utterance["text"]).cats\n  label = max(d, key =d.get)\n  utterance["label"] = label\n'

In [ ]:
#CANGED
for utterance in trial_data:
  try:
    texty = utterance["text"].replace("\\n", " ").lower()    #removing new-line-characters(\\n) and lowering the text
  except:
    texty = utterance["text"].lower()
  d = trained_nlp(texty).cats
  label = max(d, key =d.get)
  utterance["label"] = label

In [ ]:
trial_data

[{'id': '1',
  'text': 'When you wake up after being healed\nnight you got vaccinated against\ncoronavirus/I',
  'image': 'bg_memes_1.png',
  'label': 'non_propagandistic'},
 {'id': 'bg_mk_memes_984',
  'text': "What's in Kostya Kopeikin: ACE\n\nWhat's in Kiril Petkov:",
  'image': 'bg_mk_memes_984.png',
  'label': 'propagandistic'},
 {'id': '4',
  'text': 'HOW DO YOU FEEL?\nSUDDENLY I FELT\nAN UNEXPLAINABLE NEED\nTO REGISTER WINDOWS-A',
  'image': 'bg_memes_4.png',
  'label': 'non_propagandistic'},
 {'id': '41',
  'text': 'I NEVER BELIEVED THAT\nTHE BEGINNING OF THE JOKES WILL TURN OUT TO BE TRUE:\nBYE GANYO BULGARINA\nAND BYE GANYO THE AMERICAN...\nimgflip.com',
  'image': 'bg_memes_41.png',
  'label': 'non_propagandistic'},
 {'id': '49',
  'text': 'i\n2 hours after vaccination\nPIK',
  'image': 'bg_memes_49.png',
  'label': 'non_propagandistic'},
 {'id': '50',
  'text': 'BEFORE THE VACCINE\nTHE POST-VACCINE\nimgflip.com',
  'image': 'bg_memes_50.png',
  'label': 'non_propagandistic'

#Submition Format
"Subtask2b.txt", which means that the "Subtask2b.json" file should be renamed

In [ ]:
dir = "Subtask2b.json"
data = trial_data

In [ ]:
with open(dir, mode="wt", encoding = "utf-8") as f:
  json.dump(data,f,ensure_ascii=False, indent=4)